<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0d3bc97bd22bc93079bf5a42d75917ebfdb6b954b1b27e84efd01bf597dc497c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 12:58:26
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: -65.73 K (-0.45%)
Current PnL: -18.61 L (-12.2%)
CY Booked + Current PnL: -6.48 L (-4.25%)
-------------------
Total profit:  2.64 L
Total loss:  -21.25 L
-------------------
Total Booked + Current PnL: 20.50 L (16.32%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.68 L (58.85%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.91,72.0,X-LC,4.58,233282.0,22916.0,5855.0,-0.59,10.89,2.51,13.68,37.0,3.91,1.62,29.22,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,108.11,60.0,M-SC,1.29,89637.0,-11140.0,11232.0,-1.76,-11.05,12.53,0.09,245.0,-0.99,0.62,17.76,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-15.53,75.0,H-MC,17.05,217704.0,37896.0,11451.0,0.55,21.08,5.26,27.45,88.0,3.31,1.51,62.35,XR,NTT,BANKS
42,ITC,452.0,-36.66,62.0,X-LC,1.28,204989.0,4851.0,16030.0,-0.57,2.42,7.82,10.44,4.0,0.30,1.42,8.12,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.84,39.0,X-MC,2.50,88995.0,-1683.0,16589.0,-0.13,-1.86,18.64,16.44,101.0,-0.10,0.62,17.95,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,86.23,80.0,H-SC,2.50,161693.0,-21082.0,132152.0,11.28,-11.53,81.73,60.77,125.0,-0.16,1.12,44.82,XR,NTT,JEWELLERY
17,CAMS,5211.76,-2.73,56.0,X-SC,0.52,110488.0,8484.0,35445.0,2.35,8.32,32.08,43.06,122.0,0.24,0.77,26.86,X40N,ATH,MISC
50,MASFIN,398.61,-18.70,53.0,H-SC,9.17,92490.0,-5490.0,27090.0,1.62,-5.60,29.29,22.05,152.0,-0.20,0.64,34.44,XR,ATH,FINANCE
72,SYMPHONY,1306.00,-43.04,56.0,M-SC,17.84,121594.0,-49547.0,49489.0,1.51,-28.95,40.70,-0.03,196.0,-1.00,0.84,6.01,OX40N,NTT,DURABLES
65,SFL,1287.00,34.22,55.0,M-SC,6.00,177233.0,-84985.0,154813.0,1.34,-32.41,87.35,26.63,229.0,-0.55,1.23,10.19,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-33.16,47.0,H-SC,16.10,191201.0,-8753.0,105983.0,-5.22,-4.38,55.43,48.63,115.0,-0.08,1.33,6.67,XR,NTT,MISC
83,VALIANTORG,838.00,-282.16,38.0,H-SC,9.34,101150.0,-62455.0,183769.0,-3.75,-38.17,181.68,74.15,139.0,-0.34,0.70,29.66,XR,NTT,CHEMICALS
84,VBL,671.64,-14.62,60.0,X-LC,8.51,307612.0,-8230.0,120892.0,-2.68,-2.61,39.30,35.67,5.0,-0.07,2.13,10.89,X40N,ATH,FMCG
62,SAIL,228.00,49.16,59.0,M-MC,10.23,235845.0,10883.0,156766.0,-2.55,4.84,66.47,74.53,192.0,0.07,1.64,38.65,XY24,BTT,STEEL
54,QUESS,986.00,-45.45,49.0,X-SC,37.96,53907.0,-11099.0,162028.0,-2.05,-17.07,300.57,232.18,198.0,-0.07,0.37,5.15,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,-15.15,44.0,M-LC,5.69,149482.0,-1463.0,110916.0,-0.49,-0.97,74.20,72.51,53.0,-0.01,1.04,4.99,XR,NTT,IT
32,HINDZINC,730.22,27.24,48.0,M-LC,11.30,207104.0,2028.0,110552.0,-1.55,0.99,53.38,54.89,52.0,0.02,1.44,25.32,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,108.11,60.0,M-SC,1.29,89637.0,-11140.0,11232.0,-1.76,-11.05,12.53,0.09,245.0,-0.99,0.62,17.76,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,52.0,H-SC,2.83,225702.0,-43965.0,80305.0,-0.75,-16.30,35.58,13.47,138.0,-0.55,1.57,14.76,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,52.0,H-SC,2.48,141346.0,-34557.0,76581.0,0.31,-19.65,54.18,23.89,149.0,-0.45,0.98,22.07,OX40N,NTT,CERAMICS
74,TATAELXSI,9161.0,-22.37,53.0,H-MC,6.43,104966.0,-26869.0,69089.0,-0.25,-20.38,65.82,32.03,98.0,-0.39,0.73,17.57,OX40N,NTT,IT
68,SIS,528.0,2034.72,46.0,H-SC,3.20,86139.0,-24893.0,48505.0,-0.66,-22.42,56.31,21.26,156.0,-0.51,0.60,16.38,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.0,-15.53,75.0,H-MC,17.05,217704.0,37896.0,11451.0,0.55,21.08,5.26,27.45,88.0,3.31,1.51,62.35,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,49.16,59.0,M-MC,10.23,235845.0,10883.0,156766.0,-2.55,4.84,66.47,74.53,192.0,0.07,1.64,38.65,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,56.0,M-LC,5.18,208422.0,5252.0,36870.0,-1.71,2.59,17.69,20.74,55.0,0.14,1.45,9.62,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.24,48.0,M-LC,11.30,207104.0,2028.0,110552.0,-1.55,0.99,53.38,54.89,52.0,0.02,1.44,25.32,X5K,ATH,METALS
47,KPIGREEN,731.05,11.71,72.0,H-SC,5.46,133043.0,7746.0,51182.0,-0.71,6.18,38.47,47.03,141.0,0.15,0.92,66.58,MH,ATH,POWER
80,UNIONBANK,163.00,-6.98,65.0,M-LC,9.73,166923.0,26083.0,18412.0,0.47,18.52,11.03,31.59,66.0,1.42,1.16,49.91,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-15.53,75.0,H-MC,17.05,217704.0,37896.0,11451.0,0.55,21.08,5.26,27.45,88.0,3.31,1.51,62.35,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.51,66.0,M-SC,3.74,129536.0,14738.0,79302.0,-0.23,12.84,61.22,81.92,223.0,0.19,0.90,53.35,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.71,72.0,H-SC,5.46,133043.0,7746.0,51182.0,-0.71,6.18,38.47,47.03,141.0,0.15,0.92,66.58,MH,ATH,POWER
37,INDIAMART,4810.62,-53.15,57.0,H-SC,6.67,130968.0,7632.0,124001.0,-0.51,6.19,94.68,106.72,119.0,0.06,0.91,31.01,AR,ATH,MISC
32,HINDZINC,730.22,27.24,48.0,M-LC,11.30,207104.0,2028.0,110552.0,-1.55,0.99,53.38,54.89,52.0,0.02,1.44,25.32,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.04,5.24,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.15,30.0,X-SC,4.92,89582.0,-39088.0,84386.0,0.21,-30.38,94.20,35.20,219.0,-0.46,0.62,2.82,X40,NTT,FOOTWEAR
24,DMART,5391.45,-17.97,33.0,X-LC,9.67,100368.0,-2218.0,29026.0,-1.12,-2.16,28.92,26.13,38.0,-0.08,0.70,22.86,X40N,ATH,FMCG
31,HINDUNILVR,2922.00,-12.14,37.0,X-LC,7.52,246500.0,-10563.0,45701.0,-0.93,-4.11,18.54,13.67,24.0,-0.23,1.71,15.46,XY25,NTT,FMCG
45,JSWINFRA,342.00,-20.69,37.0,X-MC,5.24,199125.0,-1222.0,31721.0,-0.41,-0.61,15.93,15.23,178.0,-0.04,1.38,31.44,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.61,47.0,H-LC,1.17,156485.0,-29610.0,77085.0,-0.60,-15.91,49.26,25.51,15.0,-0.38,1.09,15.32,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,62.0,X-LC,1.28,204989.0,4851.0,16030.0,-0.57,2.42,7.82,10.44,4.0,0.30,1.42,8.12,X40,NTT,FMCG
53,PGHH,17907.65,-30.55,38.0,X-MC,2.29,200910.0,90.0,67707.0,-0.44,0.04,33.70,33.76,80.0,0.00,1.39,5.24,X40,ATH,FMCG
29,HAVELLS,2069.16,-4.76,53.0,X-MC,2.97,317877.0,-10114.0,116661.0,0.28,-3.08,36.70,32.48,92.0,-0.09,2.21,7.99,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.02,52.0,X-MC,4.20,211890.0,20148.0,17608.0,-0.44,10.51,8.31,19.69,99.0,1.14,1.47,18.02,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.15,30.0,X-SC,4.92,89582.0,-39088.0,84386.0,0.21,-30.38,94.20,35.20,219.0,-0.46,0.62,2.82,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-28.03,39.0,X-MC,6.70,81350.0,-2160.0,22192.0,-1.66,-2.59,27.28,23.98,82.0,-0.10,0.56,3.08,X40,ATH,APPARELS
3,ACC,3906.00,-37.64,52.0,X-MC,2.52,186000.0,-51951.0,204600.0,-1.03,-21.83,110.00,64.15,174.0,-0.25,1.29,4.34,XY24,BTT,CEMENT
21,COLPAL,3767.14,-3.66,49.0,X-MC,7.66,225150.0,-38215.0,151571.0,-0.71,-14.51,67.32,43.04,84.0,-0.25,1.56,4.53,XY25,ATH,FMCG
10,BAJAJHFL,181.50,-13.84,50.0,X-MC,5.88,181548.0,-18958.0,116118.0,0.05,-9.46,63.96,48.45,90.0,-0.16,1.26,4.69,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-45.45,49.0,X-SC,37.96,53907.0,-11099.0,162028.0,-2.05,-17.07,300.57,232.18,198.0,-0.07,0.37,5.15,XY24,NTT,MISC
52,PAGEIND,51769.93,-28.03,39.0,X-MC,6.70,81350.0,-2160.0,22192.0,-1.66,-2.59,27.28,23.98,82.0,-0.10,0.56,3.08,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.59,45.0,X-SC,4.77,82306.0,-62854.0,138784.0,-0.68,-43.30,168.62,52.31,136.0,-0.45,0.57,10.17,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.84,39.0,X-MC,2.50,88995.0,-1683.0,16589.0,-0.13,-1.86,18.64,16.44,101.0,-0.10,0.62,17.95,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.15,30.0,X-SC,4.92,89582.0,-39088.0,84386.0,0.21,-30.38,94.20,35.20,219.0,-0.46,0.62,2.82,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.36,52.0,X-LC,10.94,285591.0,-60365.0,127059.0,-0.63,-17.45,44.49,19.28,1.0,-0.48,1.98,5.58,X40,ATH,IT
40,INFY,2275.00,-18.52,55.0,X-LC,5.79,318116.0,5228.0,166470.0,-1.12,1.67,52.33,54.87,3.0,0.03,2.21,10.09,X40,BTT,IT
42,ITC,452.00,-36.66,62.0,X-LC,1.28,204989.0,4851.0,16030.0,-0.57,2.42,7.82,10.44,4.0,0.30,1.42,8.12,X40,NTT,FMCG
84,VBL,671.64,-14.62,60.0,X-LC,8.51,307612.0,-8230.0,120892.0,-2.68,-2.61,39.30,35.67,5.0,-0.07,2.13,10.89,X40N,ATH,FMCG
1,ABB,7934.00,-39.38,56.0,H-LC,5.60,253368.0,-8251.0,127469.0,-0.20,-3.15,50.31,45.57,7.0,-0.06,1.76,7.81,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6988.89,57.0,L-SC,7.82,77775.0,-16035.0,92653.0,-0.89,-17.09,119.13,81.67,269.0,-0.17,0.54,50.61,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.17,81120.0,-32429.0,72481.0,-0.12,-28.56,89.35,35.27,268.0,-0.45,0.56,98.54,XR,NTT,HOTELS
79,TTKPRESTIG,770.00,108.11,60.0,M-SC,1.29,89637.0,-11140.0,11232.0,-1.76,-11.05,12.53,0.09,245.0,-0.99,0.62,17.76,OX40N,NTT,DURABLES
50,MASFIN,398.61,-18.70,53.0,H-SC,9.17,92490.0,-5490.0,27090.0,1.62,-5.60,29.29,22.05,152.0,-0.20,0.64,34.44,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-37.51,66.0,M-SC,3.74,129536.0,14738.0,79302.0,-0.23,12.84,61.22,81.92,223.0,0.19,0.90,53.35,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,86.23,80.0,H-SC,2.50,161693.0,-21082.0,132152.0,11.28,-11.53,81.73,60.77,125.0,-0.16,1.12,44.82,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,11.71,72.0,H-SC,5.46,133043.0,7746.0,51182.0,-0.71,6.18,38.47,47.03,141.0,0.15,0.92,66.58,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.51,66.0,M-SC,3.74,129536.0,14738.0,79302.0,-0.23,12.84,61.22,81.92,223.0,0.19,0.90,53.35,XR,NTT,DURABLES
5,ANGELONE,3033.00,15.82,63.0,X-SC,5.56,205279.0,14273.0,43437.0,-0.33,7.47,21.16,30.21,157.0,0.33,1.42,29.91,X40N,NTT,FINANCE
69,SONACOMS,806.63,-32.20,70.0,M-SC,7.82,84455.0,-16704.0,56703.0,-0.11,-16.51,67.14,39.54,202.0,-0.29,0.59,19.09,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,43.99
2,50,74.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.11
LC    32.31
MC    24.59
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.04
X40      16.20
XY25     12.01
XR       11.48
X40N     10.66
AR        7.86
OX40N     6.56
X200      1.77
X5K       1.44
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.50
X-LC    21.61
X-MC    18.74
M-SC    12.66
M-LC     5.09
H-LC     4.62
X-SC     4.52
H-MC     3.89
M-MC     1.64
L-SC     1.43
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.48,-3.19,36.87
IT,12.47,-17.04,79.82
FINANCE,12.12,-8.25,57.31
MISC,6.83,-16.92,81.71
ELECTRICAL,5.83,-7.71,45.90
BANKS,5.67,-4.77,58.96
PAINTS,5.51,-16.28,33.62
INSURANCE,3.78,-1.46,39.30
AUTO,2.77,-44.53,104.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3246675.0,22
XR,1230792.0,14
AR,1159131.0,9
X40,819609.0,12
X40N,622922.0,9
OX40N,540029.0,9
XY25,451790.0,8
SR,266991.0,2
X5K,110552.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3305220.0,24
M-SC,1220293.0,15
X-MC,1175314.0,14
X-LC,1023262.0,13
X-SC,527937.0,6
M-LC,276750.0,4
H-LC,272990.0,3
H-MC,256606.0,3
L-SC,254625.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1156394.0      6
           AR         831169.0      5
           XR         759792.0      7
M-SC       XY24       699438.0      6
X-MC       XY24       574506.0      4
X-LC       X40        451433.0      5
           XY24       303065.0      2
X-MC       X40        283790.0      6
X-SC       X40N       281523.0      4
H-SC       SR         266991.0      2
           OX40N      239692.0      3
H-LC       AR         204554.0      2
X-LC       X40N       193560.0      3
H-MC       XY24       176066.0      1
X-MC       XY25       169179.0      2
L-SC       XR         165134.0      2
X-SC       XY24       162028.0      1
M-MC       XY24       156766.0      1
X-MC       X40N       147839.0      2
M-SC       OX40N      141757.0      4
           XY25       128962.0      1
           XR         126728.0      2
           AR         123408.0      2
M-LC       XR         110916.0      1
           X5K        110552.0      1
L-SC       OX40N       89491.0      1
X-SC       X40         84386.0      1
X-LC       XY25        75204.0      3
H-MC       OX40N       69089.0      1
H-LC       X200        68436.0      1
L-MC       XR          56771.0      1
H-SC       MH          51182.0      1
L-LC       XY25        41575.0      1
M-LC       XY25        36870.0      1
           XY24        18412.0      1
H-MC       XR          11451.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
